# Ein paar interessante Studien

### module importieren

In [ ]:
import glob

import numpy as np
import pandas as pd
from sklearn import preprocessing

from nilearn import plotting, image, input_data
from nistats import first_level_model, reporting, regression

import seaborn as sns
import matplotlib.pyplot as plt

sns.set_context('talk')

## Haxby et al. 2001 Science

Dekodieren von Hirnaktivität mittels Ähnlichkeitsmaßen

In [ ]:
from IPython.display import IFrame
IFrame("https://science.sciencemag.org/content/sci/293/5539/2425.full.pdf", width=900, height=800)

### unsere daten laden

In [ ]:
fmri_img_list = glob.glob('../data/sub-01_task-imagery_run-0*_bold_space-MNI152NLin2009cAsym_preproc.nii.gz')
fmri_img_list.sort()
fmri_img_list = fmri_img_list[1:]

### extrahieren der Daten mit einem Masker

In [ ]:
mean_img = image.mean_img([image.mean_img(f) for f in fmri_img_list])

In [ ]:
masker = input_data.NiftiMasker(standardize=True,
                                detrend=True,
                                smoothing_fwhm=6,
                                mask_strategy='epi')
masker.fit(mean_img)

In [ ]:
plotting.plot_roi(masker.mask_img_)

### jeden Run einzeln extrahieren, dann an die anderen anhängen

So erreichen wir, dass jeder Run einzeln z-transformiert wird. Dadurch ist auch der Mittlwert jedes Runs bei null und wir brauchen keine Konstante mehr.

In [ ]:
def make_big_df(fmri_img_list,masker):
    big_df = pd.DataFrame()
    for im in fmri_img_list:
        data = masker.transform(im)
        df = pd.DataFrame(data)
        big_df = pd.concat([big_df,df])
    return big_df

In [ ]:
big_df = make_big_df(fmri_img_list,masker)

In [ ]:
big_df.shape

In [ ]:
big_df.tail()

### design laden

In [ ]:
onset_csvs = glob.glob('../data/sub-01_task-imagery_run-0*_events.tsv')
onset_csvs.sort()
onset_csvs = onset_csvs[1:]

In [ ]:
def make_onsets(csv_file):
    
    df = pd.read_csv(csv_file,sep='\t')
    df.loc[:,'trial_num'] = [('0%s'%x)[-2:] for x in df.index.get_values().astype(str)]
    df.loc[:,'trial_type'] = df.loc[:,'trial_domain']
    
    return df

In [ ]:
n_volumes = image.load_img(fmri_img_list[-1]).shape[-1]
my_tr = 3
my_duration = n_volumes*my_tr

In [ ]:
def make_design(df,my_tr=my_tr,my_duration=my_duration):
    frame_times = np.arange(0,my_duration,my_tr)
    design_matrix = first_level_model.make_first_level_design_matrix(frame_times=frame_times,
                                                                     events=df,
                                                                     drift_model=None
                                                                    )
    return design_matrix

In [ ]:
def make_big_design(onset_csvs):
    
    design_matrix = pd.DataFrame()
    for run_num,onset_csv in enumerate(onset_csvs):
        this_onset = make_onsets(onset_csv)
        this_design = make_design(this_onset).iloc[:,:-1] # konstante weg
        this_design.index = this_design.index+my_duration*run_num
        this_design.columns = ['%s-run%s'%(c,run_num+1) for c in this_design.columns]

        design_matrix = pd.concat([design_matrix, this_design],sort=False)
        design_matrix = design_matrix.fillna(0)

    return design_matrix

In [ ]:
design_matrix = make_big_design(onset_csvs)

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(14,10))
sns.heatmap(design_matrix,ax=ax,cmap='Greys_r')
plt.show()

### Daten und Design

Hier multiplizieren wir den Zeitverlauf jedes Voxels mit der erwarteten HRF jeder Bedingung; so bekommen wir eine Art gewichteten Mittelwert für jede Bedingung.

In [ ]:
this_design = design_matrix.iloc[:200,4].values
this_voxel = big_df.iloc[:200,4000].values

In [ ]:
plt.figure(figsize=(16,6))
plt.plot(this_design,label='model of hrf for one condition')
plt.plot(this_voxel,label='timecourse in one voxel')
plt.xlabel('time')
plt.legend()
sns.despine()
plt.show()

In [ ]:
plt.figure(figsize=(16,6))
plt.plot(this_design * this_voxel,label='timecourse in one voxel for one condition')
plt.xlabel('time')
plt.legend(loc=(0.5,1))
sns.despine()
plt.show()

### gemittelte Aktivität für jede Bedingung

In [ ]:
def make_block_df(big_df,design_matrix):
    block_df = pd.DataFrame()
    for cond in design_matrix.columns:
        print(cond)
        f = lambda x:x*design_matrix[cond].values
        mean_block = big_df.apply(f).mean()
        mean_block_df = pd.DataFrame(mean_block).T
        mean_block_df.index = [cond]
        block_df = pd.concat([block_df, mean_block_df])
    return block_df

In [ ]:
block_df = make_block_df(big_df,design_matrix)

In [ ]:
block_df.shape

In [ ]:
block_df

Beispiel: Die Daten aus einer Zeile in den Hirnraum zurückführen

In [ ]:
block_im = masker.inverse_transform(block_df.loc['SPRACHE-run3',:])
plotting.view_img(block_im)

### Aufteilen in zwei Hälften

In [ ]:
first_half = block_df.iloc[:10,:]
first_half

In [ ]:
second_half = block_df.iloc[10:,:]
second_half

### erste Hälfte als Modell

In [ ]:
first_half.index = pd.MultiIndex.from_tuples([x.split('-') for x in first_half.index])
first_half

In [ ]:
first_half = first_half.groupby(level=0).mean()
first_half

### Alle Blocks der 2. Hälfte mit denen der 1. Hälfte Korrelieren

In [ ]:
big_corr_df = pd.DataFrame()
for i in first_half.index:
    this_corr = second_half.T.corrwith(first_half.T.loc[:,i])
    this_corr_df = pd.DataFrame(this_corr).T
    this_corr_df.index = [i]
    big_corr_df = pd.concat([big_corr_df,this_corr_df])
big_corr_df = big_corr_df.T

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(10,6))
sns.heatmap(big_corr_df,annot=True,ax=ax,cmap='RdBu_r',vmin=-1,vmax=1)
plt.show()

## Kriegeskorte et al. 2008 Neuron

Representational Similiarity Analysis (RSA). Wie ähnlich ist die "Repräsentation" verschiedener kognitiver Zustände im Gehirn bzw. bestimmten Hirnregionen?

In [ ]:
IFrame("https://www.cns.nyu.edu/kianilab/papers/Kriegeskorte_Kiani_Bandettini.pdf", width=900, height=800)

### Dissimilarity berechnen

In [ ]:
corr_df = block_df.T.corr()

In [ ]:
sns.heatmap(corr_df,cmap='RdBu_r',vmin=-1,vmax=1,square=True)

In [ ]:
diss_df = 1 - corr_df

In [ ]:
sns.heatmap(diss_df,cmap='Reds',vmin=0,vmax=2,square=True)

In [ ]:
sns.clustermap(diss_df,figsize=(16,16),cmap='Reds',annot=True)

### Multi-Dimensionale Skalierung

In [ ]:
from sklearn import manifold
from sklearn.metrics import euclidean_distances

In [ ]:
mds = manifold.MDS(n_components=2,
                   dissimilarity="precomputed")

In [ ]:
mds_positions = mds.fit( diss_df ).embedding_

In [ ]:
mds_df = pd.DataFrame(mds_positions,index=diss_df.index)

In [ ]:
mds_df

In [ ]:
mds_df.loc[:,'group'] = [x.split('-')[0] for x in mds_df.index]

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(12,12))
sns.scatterplot(data=mds_df,x=0,y=1,hue='group',ax=ax)
for i in mds_df.index:
    x,y = mds_df.loc[i,[0,1]]
    plt.annotate(s=i,xy=(x,y))
#plt.xticks([]); plt.yticks([])
#plt.xlabel(''); plt.ylabel('')
#sns.despine(left=True,bottom=True)
plt.show()

## Hasson et al. 2004 Science

Reverse Correlation: Welche Art von Stimulus aktiviert eine Hirnregion maximal?

In [ ]:
IFrame("http://www.psiexp.ss.uci.edu/research/teachingP140C/Papers/hasson_2004.pdf", width=900, height=800)

### Daten in Hirnraum zurückführen und Signal aus Koordinate extrahieren

In [ ]:
big_im = masker.inverse_transform(big_df)

In [ ]:
sphere_masker = input_data.NiftiSpheresMasker([(-52,12,28)],
                                              radius=5,
                                              t_r=my_tr,
                                              low_pass=0.1).fit()

In [ ]:
sphere_data = sphere_masker.transform(big_im)

In [ ]:
sphere_df = pd.DataFrame(sphere_data)
sphere_df.columns = [','.join([str(i) for i in x]) for x in sphere_masker.seeds_]
sphere_df.loc[:,'time'] = sphere_df.index*my_tr

In [ ]:
sphere_df.tail()

In [ ]:
col_name = sphere_df.columns[0]
col_name

In [ ]:
sphere_data.shape

### Aktivität in der extrahieren Region über die Zeit

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(16,6))
ax.plot(sphere_df.loc[:,col_name])
plt.axhline(0,color='k')
plt.xlabel('time')
plt.ylabel('z-transformed signal')
sns.despine()
plt.show()

Höchste Werte

In [ ]:
my_max_values = sphere_df.loc[:,col_name].sort_values(ascending=False)[:10].values
my_max_values

Index der höchsten Werte

In [ ]:
my_max_idx = sphere_df.loc[:,col_name].sort_values(ascending=False)[:10].index
my_max_idx

Zeit in Sekunden der höchsten Werte

In [ ]:
my_max_time = [ sphere_df.loc[i,'time'] for i in my_max_idx]
my_max_time

Inhalt der Design-Matrix zu diesen Zeitpunkten

In [ ]:
my_max_label = [design_matrix.loc[i].idxmax() for i in my_max_time]
my_max_label

### Annotierter Zeitverlauf

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(16,6))
ax.plot(sphere_df.loc[:,col_name],alpha=0.9)
for (s,x,y) in zip(my_max_label,my_max_idx,my_max_values):
    plt.plot(x,y,'o',color='r',alpha=0.5)
    plt.annotate(s=s,xy=(x,y))
plt.axhline(0,color='k')
plt.xlabel('time')
plt.ylabel('z-transformed signal')
sns.despine()
plt.show()

### DIY fMRI

In [ ]:
IFrame('https://neuwritesd.org/2015/07/10/got-a-computer-we-have-data-lets-do-neuroscience/', width=700, height=350)

In [ ]:
IFrame('https://antmelder.files.wordpress.com/2014/06/tumblr_m4j2quesvw1qedj2ho1_1280.jpg', width=700, height=500)